# LangChain 卡牌效果生成器

本 notebook 演示如何使用 LangChain 和大语言模型来生成卡牌效果。

In [ ]:
# 导入必要的库
from typing import Dict, List, Any, Union, Optional
from langchain.prompts import PromptTemplate
from langchain.output_parsers import PydanticOutputParser
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain
from pydantic import BaseModel, Field, validator
import os

## 1. 设置环境变量
请确保设置了您的 OpenAI API 密钥

In [ ]:
# 设置 OpenAI API 密钥
# os.environ["OPENAI_API_KEY"] = "your-api-key-here"

# 初始化 LLM
llm = ChatOpenAI(
    temperature=0.7,
    model_name="gpt-3.5-turbo"
)

## 2. 定义数据模型

In [ ]:
class CardEffect(BaseModel):
    effect_type: str = Field(..., description="效果类型")
    target_type: str = Field(..., description="目标类型")
    value: Union[int, str] = Field(..., description="效果值")
    condition: Optional[str] = Field(None, description="触发条件")
    duration: Optional[int] = Field(None, description="持续回合数")

class CommandOutput(BaseModel):
    card_id: str = Field(..., description="卡牌ID")
    effect: CardEffect = Field(..., description="卡牌效果")
    description: str = Field(..., description="卡牌描述")

## 3. 设置提示模板和输出解析器

In [ ]:
# 创建输出解析器
parser = PydanticOutputParser(pydantic_object=CommandOutput)

# 创建提示模板
template = """根据以下卡牌信息生成卡牌效果：
卡牌名称: {card_name}
卡牌类型: {card_type}
费用: {cost}

请生成一个符合游戏平衡性的卡牌效果。

{format_instructions}
"""

prompt = PromptTemplate(
    template=template,
    input_variables=["card_name", "card_type", "cost"],
    partial_variables={"format_instructions": parser.get_format_instructions()}
)

## 4. 创建 LangChain 链

In [ ]:
# 创建 LangChain 链
chain = LLMChain(
    llm=llm,
    prompt=prompt,
    verbose=True
)

## 5. 生成卡牌效果示例

In [ ]:
# 示例卡牌信息
card_info = {
    "card_name": "炎魔术士",
    "card_type": "随从",
    "cost": 4
}

# 运行链并获取结果
response = chain.run(card_info)

# 解析响应
try:
    card_output = parser.parse(response)
    print("生成的卡牌效果：")
    print(f"卡牌ID: {card_output.card_id}")
    print(f"效果类型: {card_output.effect.effect_type}")
    print(f"目标类型: {card_output.effect.target_type}")
    print(f"效果值: {card_output.effect.value}")
    print(f"描述: {card_output.description}")
except Exception as e:
    print(f"解析错误: {e}")
    print(f"原始响应: {response}")